In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.mars_db
collection = db.articles

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='list_text')

# loop over results to get article data
for result in results:
    
    # scrape title
    news_title = result.find('div', class_='content_title').text
    
    # scrape paragraph text
    news_p = result.find('div', class_='article_teaser_body').text
    
    # print article data
    print(news_title)
    print(news_p)